In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pm4py
from pm4py.algo.discovery.dcr_discover import algorithm as alg
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.objects.dcr.semantics import DcrSemantics
from pm4py.objects.conversion.dcr import *
import pandas as pd

# Loading data

In [3]:
el = pm4py.read_xes('/home/vco/Projects/pm4py-dcr/datasets/rtfmp.xes',return_legacy_log_object=True)
el_df = pm4py.read_xes('/home/vco/Projects/pm4py-dcr/datasets/rtfmp.xes')
temp_df = el_df[['case:concept:name','concept:name','time:timestamp']].sort_values(['concept:name'])
temp_df[temp_df['case:concept:name'] == 'S100337'].sort_values(['case:concept:name','time:timestamp'])
old_ef_format = pm4py.convert_to_event_log(el_df)

/home/vco/.pyenv/versions/3.11.0/envs/pm4py-dcr/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 150370/150370 [00:16<00:00, 9161.01it/s] 
/home/vco/Projects/pm4py-dcr/pm4py/utils.py:494: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


# Algorithm examples

In [4]:
model, _ = alg.apply(old_ef_format,variant=alg.DCR_BASIC)
model_sp, _ = alg.apply(el,variant=alg.DCR_SUBPROCESS)

[i] Mining with basic DisCoveR
[i] Mining with Sp-DisCoveR


/home/vco/Projects/pm4py-dcr/pm4py/utils.py:494: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [ ]:
params = {'timed': True}
model_t, _ = alg.apply(el,variant=alg.DCR_BASIC, **params)
params = {'pending': True}
model_p, _ = alg.apply(el,variant=alg.DCR_BASIC, **params)

# Conversion examples

# Import export execution semantics objects
Workflow:
* Import an xml downloaded from the dcr portal.
* Play with the execution semantics (time and subprocesses are supported)
* Export an xml simple and import it into the dcr portal.


In [5]:
dcr_graph = dcr_importer.apply('data/dcr_from_portal.xml')

In [6]:
dcr_graph

{'events': {'A', 'B', 'C', 'D'},
 'labels': {'A', 'B', 'C', 'D'},
 'labelMapping': {'A': {'A'}, 'C': {'C'}, 'D': {'D'}, 'B': {'B'}},
 'conditionsFor': {'B': {'C'}},
 'milestonesFor': {'B': {'A'}},
 'responseTo': {'B': {'A'}},
 'noResponseTo': {'D': {'B'}},
 'includesTo': {'C': {'D'}},
 'excludesTo': {'D': {'D'}},
 'conditionsForDelays': {},
 'responseToDeadlines': {},
 'marking': {'executed': set(),
  'included': {'A', 'B', 'C', 'D'},
  'pending': set()},
 'subprocesses': {}}

In [7]:
semantics_obj = DcrSemantics(dcr_graph)
semantics_obj.execute('C')
semantics_obj.execute(31)
semantics_obj.enabled()

{'A', 'B', 'C', 'D'}

In [8]:
semantics_obj.find_next_deadline()

In [9]:
semantics_obj.find_next_delay()

In [10]:
dcr_exporter.apply(dcr_graph,'data/dcr_to_portal.xml')